In [ ]:
%load_ext dockermagic

# HDFS

## HDFS - Web Interface

- Master node
    - NameNode: http://localhost:9870
    - Secondary NameNode: http://localhost:9868
- Worker node
    - hadoop1
        - DataNode: http://localhost:9864
    - hadoop2
        - DataNode: http://localhost:9865
    - hadoop3
        - DataNode: http://localhost:9866

## Filesystem Basic Commands

- https://hadoop.apache.org/docs/stable/hadoop-project-dist/hadoop-common/FileSystemShell.html

Download books from Gutenberg project (http://www.gutenberg.org/)

- Moby Dick; Or, The Whale by Herman Melville
- Pride and Prejudice by Jane Austen
- Dracula by Bram Stoker

In [ ]:
%%dockerexec hadoop

source /opt/envvars.sh
mkdir /opt/datasets
cd /opt/datasets

wget -qc http://www.gutenberg.org/files/2701/2701-0.txt -O mobydick.txt
wget -qc http://www.gutenberg.org/files/1342/1342-0.txt -O prideandprejudice.txt
wget -qc http://www.gutenberg.org/cache/epub/345/pg345.txt -O dracula.txt

ls /opt/datasets

### Create gutenberg folder in HDFS

In [ ]:
%%dockerexec hadoop

source /opt/envvars.sh

cd /opt/datasets

hdfs dfs -mkdir /user/hadoop/gutenberg

### Copy books to HDFS

In [ ]:
%%dockerexec hadoop

source /opt/envvars.sh

cd /opt/datasets

hdfs dfs -put * /user/hadoop/gutenberg
# hdfs dfs -copyFromLocal * /user/hadoop/gutenberg

### List files in HDFS

In [ ]:
%%dockerexec hadoop

source /opt/envvars.sh

cd /opt/datasets

hdfs dfs -ls /user/hadoop/gutenberg

### Show first/last KB of file

In [ ]:
%%dockerexec hadoop

source /opt/envvars.sh

cd /opt/datasets

# hdfs dfs -head /user/hadoop/gutenberg/mobydick.txt
hdfs dfs -tail /user/hadoop/gutenberg/prideandprejudice.txt

### Show whole file - CAREFUL

In [ ]:
%%dockerexec hadoop

source /opt/envvars.sh

cd /opt/datasets

hdfs dfs -cat /user/hadoop/gutenberg/dracula.txt

### Append file contents to a file in HDFS

In [ ]:
%%dockerexec hadoop

source /opt/envvars.sh

cd /opt/datasets

hdfs dfs -appendToFile mobydick.txt prideandprejudice.txt dracula.txt /user/hadoop/allbooks.txt

### Copy allbooks.txt (in HDFS) to gutenberg directory (in HDFS)


In [ ]:
%%dockerexec hadoop

source /opt/envvars.sh

cd /opt/datasets

#hdfs dfs -cp allbooks.txt /user/hadoop/gutenberg
hdfs dfs -ls -h -R /user/hadoop/gutenberg

### Copy allbooks.txt to local filesystem

In [ ]:
%%dockerexec hadoop

source /opt/envvars.sh

cd /opt/datasets

hdfs dfs -get allbooks.txt .
# hdfs dfs -copyToLocal /user/hadoop/allbooks.txt .
ls -l allbooks.txt
rm allbooks.txt

### Remove file in HDFS

In [ ]:
%%dockerexec hadoop

source /opt/envvars.sh

cd /opt/datasets

hdfs dfs -rm allbooks.txt
# hdfs dfs -rm /user/hadoop/allbooks.txt


### Move file in HDFS (also used for renaming)

In [ ]:
%%dockerexec hadoop

source /opt/envvars.sh

cd /opt/datasets

hdfs dfs -mv gutenberg/allbooks.txt gutenberg/books.txt

### Print statistics on folder

In [ ]:
%%dockerexec hadoop

source /opt/envvars.sh

cd /opt/datasets

printf "name\ttype\tsize\treps\n"
hdfs dfs -stat "%n %F %b %r" /user/hadoop/gutenberg/*

### Get several files from HDFS and merge to a single local file

In [ ]:
%%dockerexec hadoop

source /opt/envvars.sh

cd /opt/datasets

hdfs dfs -getmerge /user/hadoop/gutenberg mergebooks.txt
ls -l mergebooks.txt
rm mergebooks.txt

### Remove directory and files (-R recursive)

In [ ]:
%%dockerexec hadoop

source /opt/envvars.sh

cd /opt/datasets

# hdfs dfs -rm -R /user/hadoop/gutenberg
hdfs dfs -ls /user/hadoop

## Utilization in a MapReduce job

### Copy files to HDFS

In [ ]:
%%dockerexec hadoop

source /opt/envvars.sh
cd /opt/datasets

hdfs dfs -mkdir /user/hadoop/gutenberg
hdfs dfs -put mobydick.txt prideandprejudice.txt dracula.txt /user/hadoop/gutenberg

### Run MapReduce application specifying HDFS folders for input and output files

In [ ]:
%%dockerexec hadoop

source /opt/envvars.sh
cd /opt/hadoop/share/hadoop/mapreduce

# run wordcount application
hadoop jar ./hadoop-mapreduce-examples-$HADOOP_VERSION.jar wordcount \
/user/hadoop/gutenberg /user/hadoop/gutenberg-output

### Show output files

In [ ]:
%%dockerexec hadoop

source /opt/envvars.sh

hdfs dfs -ls /user/hadoop/gutenberg-output
hdfs dfs -head /user/hadoop/gutenberg-output/part-r-00000

### Copy HDFS files to local filesystem

In [ ]:
%%dockerexec hadoop

source /opt/envvars.sh

cd /tmp

hdfs dfs -get /user/hadoop/gutenberg-output/part-r-00000 gutenberg-output.txt
head /tmp/gutenberg-output.txt

### Remove output folder on HDFS

In [ ]:
%%dockerexec hadoop

source /opt/envvars.sh

hdfs dfs -rm -R /user/hadoop/gutenberg-output

### Running MapReduce with 2 reduce tasks (-Dmapreduce.job.reduces=2)

In [ ]:
%%dockerexec hadoop

source /opt/envvars.sh

cd /opt/hadoop/share/hadoop/mapreduce

# run wordcount application with 2 reducers
hadoop jar ./hadoop-mapreduce-examples-$HADOOP_VERSION.jar wordcount \
-Dmapreduce.job.reduces=2 \
/user/hadoop/gutenberg /user/hadoop/gutenberg-output

### List output folder contents

In [ ]:
%%dockerexec hadoop

source /opt/envvars.sh

hdfs dfs -ls /user/hadoop/gutenberg-output

### Copy HDFS file to local filesystem

In [ ]:
%%dockerexec hadoop

source /opt/envvars.sh

cd /tmp

hdfs dfs -getmerge /user/hadoop/gutenberg-output gutenberg-output.txt
head /tmp/gutenberg-output.txt

# remove output folder
hdfs dfs -rm -R /user/hadoop/gutenberg-output

## WebHDFS

- hdfs-site.xml
    dfs.webhdfs.enabled = true

https://hadoop.apache.org/docs/stable/hadoop-project-dist/hadoop-hdfs/WebHDFS.html

**LISTSTATUS** - List a Directory

In [ ]:
%%dockerexec hadoop

curl -s http://localhost:9870/webhdfs/v1/tmp?op=LISTSTATUS | python3 -m json.tool

**CREATE** - Create and Write to a File

In [ ]:
%%dockerexec hadoop

cd /opt/datasets

# -L option -> follows redirects
curl -s -X PUT -T mobydick.txt -L "http://localhost:9870/webhdfs/v1/tmp/mobydick.txt?op=CREATE&user.name=hadoop"
curl -s "http://localhost:9870/webhdfs/v1/tmp?op=LISTSTATUS" | python3 -m json.tool

**GETFILESTATUS** - Status of a File/Directory

In [ ]:
%%dockerexec hadoop

curl -s "http://localhost:9870/webhdfs/v1/tmp/mobydick.txt?op=GETFILESTATUS" | python3 -m json.tool

**GETFILEBLOCKLOCATIONS** - Get File Block Locations

In [ ]:
%%dockerexec hadoop

curl -s "http://localhost:9870/webhdfs/v1/tmp/mobydick.txt?op=GETFILEBLOCKLOCATIONS" | python3 -m json.tool

**DELETE** - Delete a File/Directory

In [ ]:
%%dockerexec hadoop

curl -s -X DELETE "http://localhost:9870/webhdfs/v1/tmp/mobydick.txt?op=DELETE"
curl -s "http://localhost:9870/webhdfs/v1/tmp?op=LISTSTATUS" | python3 -m json.tool